In [ ]:
import requests
import csv

# Replace with your GitHub username and repository name
access_token = input()
username = 'evidence-dev'
repository = 'evidence'

# Define the GitHub API endpoint for stargazers
endpoint = f'https://api.github.com/repos/{username}/{repository}/stargazers'

# Initialize a list to store user details
stargazer_details = []
count=1

# Create a function to paginate through the stargazers
def get_stargazers(url,count):
    response = requests.get(url, headers={'Authorization': f'token {access_token}'})
    if response.status_code == 200:
        stargazers = response.json()
        for stargazer in stargazers:
            print(count)
            count+=1
            stargazer_url = stargazer['url']
            user_info = requests.get(stargazer_url, headers={'Authorization': f'token {access_token}'})
            if user_info.status_code == 200:
                stargazer_details.append(user_info.json())
        if 'next' in response.links:
            get_stargazers(response.links['next']['url'],count)

# Start the process
get_stargazers(endpoint,count)

In [ ]:
# Save stargazer details to a Parquet file

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df = pd.json_normalize(stargazer_details)
parquet_file = '../sources/stargazer_details.parquet'
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_file)

print(f"Stargazer details have been saved to {parquet_file}")